# The Tractable Buffer Stock Model

In [ ]:
# Initial setup
%matplotlib inline
import matplotlib.pyplot as plt

import sys 
import os
sys.path.insert(0, os.path.abspath('../lib'))

import numpy as np
import HARK # Prevents import error from Demos repo
from time import clock
from copy import deepcopy
mystr = lambda number : "{:.3f}".format(number)

plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('Dark2')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from HARK.utilities import plotFuncs
from HARK.ConsumptionSaving.TractableBufferStockModel import TractableConsumerType
import HARK.ConsumptionSaving.ConsumerParameters as Params

The [TractableBufferStock](http://www.econ2.jhu.edu/people/ccarroll/public/LectureNotes/Consumption/TractableBufferStock/) model is a (relatively) simple framework that captures all of the qualitative, and many of the quantitative featurs of optimal consumption in the presence of labor income uncertainty.  

The key assumption that yields tractability is that there is only a single, particularly stark form of uncertainty.  While an employed consumer remains employed, his labor income will rise at a known (for convenience, constant) rate.  But at any period there is a risk of becoming "unemployed" which is an absorbing state, like retirement or disability.

A consumer with CRRA utility $U(C) = \frac{C^{1-\rho}}{1-\rho}$ solves an optimization problem that looks standard:

\begin{eqnarray*}
V_t(M_t) &=& \max_{C_t} U(C_t) + \beta \left((1-p)V_{t+1}^{e}(M_{t+1})+p V_{t+1}^{u}(M_{t+1})\right), \\
A_t &=& M_t - C_t, \\
M_{t+1} &=& R A_t + \mathbb{1}(Y_{t+1}), \\
Y_{t+1} &=& \Gamma_{t+1} Y_t,
\end{eqnarray*}

where $\mathbb{1}$ is an indicator of whether the consumer is employed in the next period.

This model exhibits a target level of wealth with an analytical solution that exhibits plausible relationships among all of the parameters of the model.  (See the linked handout for details).

#### Fig 1. Phase Diagram

From equations (47) and (51) in the notes, we know that the steady state levels of $m^e$ (market resources when employed), and $c^e$ (consumption when employed), are defined, respectively, by the following two equations:

\begin{eqnarray*}
c^e &=&(1-R^{-1})m^e + R^{-1}, \\
c^e &=& \frac{Rkm^e\Pi}{(1+Rk\Pi)}
\end{eqnarray*}

We can use these equations to plot a phase diagram

In [ ]:
# Plot phase diagram

# Define the functions we need to compute from the above two equations

def MPCPerfForesight(R, Beta, Rho): # k is the MPC for the perfect foresight consumer in the notes
    k = 1- (R**(-1))*(R*Beta)**(1/Rho)
    return k

def PatienceFactor(R, Beta, Rho, G, Upsilon):
    Gamma = G/(1-Upsilon)
    PatienceFactor = ((R*Beta)**(1/Rho))/Gamma
    return PatienceFactor
    
def PiFunc(R, Beta, Rho, G, Upsilon):
    Gamma = G/(1-Upsilon)
    PatFac = PatienceFactor(R, Beta, Rho, G, Upsilon)
    Pi = ((PatFac**(-Rho)-1+Upsilon)/Upsilon)**(1/Rho)
    return Pi
    
def mEDelEqZero(R, mE): # Function that gives us locus of where change in mE is zero
    cE = (1-R**(-1))*mE + R**(-1)
    return cE

def cEDelEqZero(R, k, Pi, mE): # Function that gives us locus of where change in cE is zero
    cE = (R*k*mE*Pi)/(1+R*k*Pi)
    return cE

# Define a function that plots the figure, given some inputs
def Plot1(R, Beta, Rho, G, Upsilon, mEMax, mEMin):
    '''
    Plots the phase diagram for the tractable buffer stock model       

    Parameters
    ----------
    R : float
        Risk free rate of return
    Beta : float
        Discount factor
    Rho : float
        Coefficient of relative risk aversion
    G : float
        Labour income growth
    Upsilon: float
        Constant probability of becoming unemployed
    mE : np.array
        Array of market wealth values to plot
    mMax : float
       Max mE
    mMin : 
       Min mE
       
    Returns
    -------
    None
    '''
    # Do some initial working out of some functions
    k = MPCPerfForesight(R, Beta, Rho)
    Pi = PiFunc(R, Beta, Rho, G, Upsilon)
    
    # Define the range of mE to plot over
    mE_range = np.linspace(mEMin, mEMax, 1000)
    
    # Plot the stable loci
    plt.figure(figsize=(9,6))
    plt.plot(mE_range, mEDelEqZero(R, mE_range), 'k-', label='$\Delta m^e = 0$') # Plot the first equation
    plt.plot(mE_range, cEDelEqZero(R, k, Pi, mE_range), 'b-', label='$\Delta c^e = 0$') # Plot the second equation
    
    # Find the stable arm (i.e. the consumption function)
    # Copy a dictionary from a perfect foresight consumer, we will change these parameters below
    TBS_dictionary = {
        'CRRA': 2.5,          # The dictionary includes our original parameters...
        'Rfree': 1.,
        'DiscFac': 1.,
        'LivPrb': [1.0],
        'PermGroFac': [1.0],
        'PermShkStd': [0.],   # no permanent shocks for our agent...    
        'PermShkCount': 1.,
        'TranShkStd': [0.],
        'TranShkCount': 0,
        'UnempPrb': 0.0,
        'IncUnemp': 0.,
        'BoroCnstArt': 0.0,
        'aXtraMin': 0.001,    # aXtra parameters specify how to construct the grid of assets.
        'aXtraMax': 0.5,      # Don't worry about these for now
        'aXtraNestFac': 3,
        'aXtraCount': 0,
        'aXtraExtra': [None],
        'vFuncBool': False,   # These booleans indicate whether the value function should be calculated
        'CubicBool': False,   # and whether to use cubic spline interpolation. You can ignore them.
        'aNrmInitMean' : 0.,
        'aNrmInitStd' : 0.0,  # These parameters specify the (log) distribution of normalized assets
        'pLvlInitMean' : 1.5, # and permanent income for agents at "birth". They are only relevant in
        'pLvlInitStd' : 0.,   # simulation and you don't need to worry about them.
        'PermGroFacAgg' : 0.,
        'T_retire': 0,        
        'UnempPrbRet': 0.0,   
        'IncUnempRet': 0.0, 
        'T_age' : None,
        'T_cycle' : 1,
        'cycles' : 0, #consumer is infinitely lived
        'AgentCount': 1,
        'tax_rate':0.0,
    }
    
    # Change the parameters we want
    MyTBStype = TractableConsumerType(**TBS_dictionary)
    MyTBStype.UnempPrb = Upsilon
    MyTBStype.DiscFac = Beta
    MyTBStype.Rfree = R
    MyTBStype.PermGroFac = G/(1-Upsilon)
    MyTBStype.CRRA = Rho

    try:
        MyTBStype.solve()
    except:
        print('Those parameter values violate a condition required for solution!')
    cE = MyTBStype.solution[0].cFunc(mE_range)*MyTBStype.PermGroFac#Cmp#/(1-Upsilon)   
    
    plt.plot(mE_range, cE, 'g--', label='Stable arm') # Plot the stable arm
    
    plt.ylim(0, 1.5*cEDelEqZero(R, k, Pi, mEMax))
    plt.xlim(mEMin, mEMax)
    plt.xlabel('$m^e_{t}$')
    plt.ylabel('$c^e_{t}$')
    plt.legend()
    plt.show()
    
    return None

# Define some functions that will serve as widgets in our interactive plot

R_widget1 = widgets.FloatSlider(
    min=1.,
    max=1.03,
    step=0.001,
    value=1.01,
    continuous_update=False,
    readout_format='.4f',
    description='$R$')

Beta_widget1 = widgets.FloatSlider(
    min=0.8,
    max=0.99,
    step=0.001,
    value=0.96,
    continuous_update=False,
    readout_format='.3f',
    description='$\\beta$')

Rho_widget1 = widgets.FloatSlider(
    min=0.1,
    max=3,
    step=0.01,
    value=1.1,
    continuous_update=False,
    readout_format='.3f',
    description='$\\rho$')

G_widget1 = widgets.FloatSlider(
    min=1.,
    max=1.02,
    step=0.001,
    value=1.0025,
    continuous_update=False,
    readout_format='.3f',
    description='$G$')

Upsilon_widget1 = widgets.FloatSlider(
    min=0.0001,
    max=0.01,
    step=0.001,
    value=0.005,
    continuous_update=False,
    readout_format='.3f',
    description='$\mho$')

mEMax_widget1 = widgets.FloatText(
    value=10,
    step=0.1,
    description='$m^e max$',
    disabled=False)

mEMin_widget1 = widgets.FloatText(
    value=0.,
    step=0.1,
    description='$m^e min$',
    disabled=False)

# Make the widget
interact(Plot1,
         R = R_widget1,
         Beta = Beta_widget1,
         Rho = Rho_widget1,
         G= G_widget1,
         Upsilon = Upsilon_widget1,
         mEMax = mEMax_widget1,
         mEMin = mEMin_widget1
        );

#### Fig 2. The Consumption Function

We can use the HARK framework to solve the consumer's problem and plot the consumption function.

In [ ]:
# Plot the consumption functions for both the buffer stock type, and perfect foresight type

# Define a function that gives us the perfect foresight consumer's consumption function
# For ease we will use the solution in the notes, but we could use HARK if we wanted here
def PerfForesightC(R, Beta, Rho, G, mE):
    k = MPCPerfForesight(R, Beta, Rho)
    h = 1/(1-G/R)
    cE = (mE-1+h)*k
    return cE

# Define a function that plots something given some inputs
def Plot2(R, Beta, Rho, G, Upsilon, mEMin, mEMax, plot_perf, plot_45, plot_emp, plot_ret, plot_mSS, show_targ):
    
    # Define the range of mE to plot over
    mE_range = np.linspace(mEMin, mEMax, 1000)
    
    # Plot the perfect foresight consumption function
    plt.figure(figsize=(9,6))
    if plot_perf:
        plt.plot(mE_range, PerfForesightC(R, Beta, Rho, G, mE_range), 'b-', label='Perfect Foresight Consumption Function')
    
    if plot_45:
        plt.plot(mE_range, mE_range, 'k-', label='45 Degree Line')
    
    # Find the consumption function
    # Copy a dictionary from a perfect foresight consumer, we will change these parameters below
    TBS_dictionary = {
        'CRRA': 2.5,          # The dictionary includes our original parameters...
        'Rfree': 1.,
        'DiscFac': 1.,
        'LivPrb': [1.0],
        'PermGroFac': [1.0],
        'PermShkStd': [0.],   # no permanent shocks for our agent...    
        'PermShkCount': 1.,
        'TranShkStd': [0.],
        'TranShkCount': 0,
        'UnempPrb': 0.0,
        'IncUnemp': 0.,
        'BoroCnstArt': 0.0,
        'aXtraMin': 0.001,    # aXtra parameters specify how to construct the grid of assets.
        'aXtraMax': 0.5,      # Don't worry about these for now
        'aXtraNestFac': 3,
        'aXtraCount': 0,
        'aXtraExtra': [None],
        'vFuncBool': False,   # These booleans indicate whether the value function should be calculated
        'CubicBool': False,   # and whether to use cubic spline interpolation. You can ignore them.
        'aNrmInitMean' : 0.,
        'aNrmInitStd' : 0.0,  # These parameters specify the (log) distribution of normalized assets
        'pLvlInitMean' : 1.5, # and permanent income for agents at "birth". They are only relevant in
        'pLvlInitStd' : 0.,   # simulation and you don't need to worry about them.
        'PermGroFacAgg' : 0.,
        'T_retire': 0,        
        'UnempPrbRet': 0.0,   
        'IncUnempRet': 0.0, 
        'T_age' : None,
        'T_cycle' : 1,
        'cycles' : 0, #consumer is infinitely lived
        'AgentCount': 1,
        'tax_rate':0.0,
    }
    
    # Change the parameters we want
    MyTBStype = TractableConsumerType(**TBS_dictionary)
    MyTBStype.UnempPrb = Upsilon
    MyTBStype.DiscFac = Beta
    MyTBStype.Rfree = R
    MyTBStype.PermGroFac = G/(1-Upsilon)
    MyTBStype.CRRA = Rho

    try:
        MyTBStype.solve()
    except:
        print('Those parameter values violate a condition required for solution!')
    cE = MyTBStype.solution[0].cFunc(mE_range)*MyTBStype.PermGroFac#Cmp#/(1-Upsilon)
    cU = MyTBStype.solution[0].cFunc_U(mE_range)*MyTBStype.PermGroFac#Cmp#/(1-Upsilon)
    
    if plot_emp:
        plt.plot(mE_range, cE, 'g-', label='Employed Consumption Function') # Plot the consumption function for the employed consumer
    
    if plot_ret:
        plt.plot(mE_range, cU, 'g--', label='Unemployed Consumption Function') # Plot the consumption function for the unemployed consumer
    
    if plot_mSS:
        plt.plot([mEMin,mEMax],
                 [(MyTBStype.PermGroFacCmp/MyTBStype.Rfree + mEMin*(1.0-MyTBStype.PermGroFacCmp/MyTBStype.Rfree)),
                  (MyTBStype.PermGroFacCmp/MyTBStype.Rfree + mEMax*(1.0-MyTBStype.PermGroFacCmp/MyTBStype.Rfree))],'--k',
                label='Sustainable C line')
    if show_targ:
        mTarg = MyTBStype.mTarg
        cTarg = MyTBStype.cTarg
        targ_label = '$m^* =$' + mystr(mTarg) + '\n$c^* =$' + mystr(cTarg)
        plt.annotate(targ_label,xy=(0.0,0.0),xytext=(0.8,0.05),textcoords='axes fraction')
    
    plt.xlim(mEMin, mEMax)
    plt.ylim(0, mEMax*0.1)
    plt.xlabel('$m^e_{t}$')
    plt.ylabel('$c^e_{t}$')
    plt.legend()
    plt.show()
    
    return None


# Define some functions that will serve as widgets in our interactive plot

R_widget2 = widgets.FloatSlider(
    min=1.,
    max=1.03,
    step=0.001,
    value=1.02,
    continuous_update=False,
    readout_format='.4f',
    description='$R$')

Beta_widget2 = widgets.FloatSlider(
    min=0.8,
    max=0.99,
    step=0.001,
    value=0.96,
    continuous_update=False,
    readout_format='.3f',
    description='$\\beta$')

Rho_widget2 = widgets.FloatSlider(
    min=0.1,
    max=3,
    step=0.01,
    value=1.1,
    continuous_update=False,
    readout_format='.3f',
    description='$\\rho$')

G_widget2 = widgets.FloatSlider(
    min=1.,
    max=1.02,
    step=0.001,
    value=1.0025,
    continuous_update=False,
    readout_format='.3f',
    description='$G$')

Upsilon_widget2 = widgets.FloatSlider(
    min=0.0001,
    max=0.01,
    step=0.001,
    value=0.005,
    continuous_update=False,
    readout_format='.3f',
    description='$\mho$')

mEMax_widget2 = widgets.FloatText(
    value=60,
    step=1,
    description='$m^e max$',
    disabled=False)

mEMin_widget2 = widgets.FloatText(
    value=0.,
    step=0.1,
    description='$m^e min$',
    disabled=False)

plot_emp_widget = widgets.Checkbox(
    value=True,
    description='Plot employed $C$ function',
    disabled=False)

plot_45_widget = widgets.Checkbox(
    value=True,
    description='Plot 45 degree line',
    disabled=False)

plot_perf_widget = widgets.Checkbox(
    value=True,
    description='Plot perfect foresight consumption function',
    disabled=False)

plot_ret_widget = widgets.Checkbox(
    value=True,
    description='Plot retired $C$ function',
    disabled=False)

plot_mSS_widget = widgets.Checkbox(
    value=True,
    description='Plot sustainable $C$ line',
    disabled=False)

show_targ_widget = widgets.Checkbox(
    value=True,
    description = 'Show target $(M,C)$',
    disabled = False)


# Make the widget
interact(Plot2,
         R = R_widget2,
         Beta = Beta_widget2,
         Rho = Rho_widget2,
         G = G_widget2,
         Upsilon = Upsilon_widget2,
         mEMax = mEMax_widget2,
         mEMin = mEMin_widget2,
         show_targ = show_targ_widget,
         plot_emp = plot_emp_widget,
         plot_ret = plot_ret_widget,
         plot_mSS = plot_mSS_widget,
         plot_45 = plot_45_widget,
         plot_perf = plot_perf_widget
        );

#### Fig. 3 Income and Consumption Growth

Now we want to depict the growth rate of consumption as a function of $m^e$. We can approximate this using the change in log consumption, the formula for which is given as equation (58) in the notes.

Moreover, we note that the target value of market resources, $\breve{m}^e$, is equal to the point where consumption from equals the log value of $\Gamma$.

Lastly, we also plot the log of the patience factor (the thorn symbol), which allows us to see the precautionary saving increment: the different between the consumption growth line and the horizontal patience factor line.

In [ ]:
# Plot the income and consumption growth chart

def Plot3(R, Beta, Rho, G, Upsilon, mEMax, mEMin):
    
    # Define the range of mE to plot over
    mE_range = np.linspace(mEMin, mEMax, 1000)
    
    # Get the patience factor
    PatFac = PatienceFactor(R, Beta, Rho, G, Upsilon)
    
    # Find the consumption function
    # Copy a dictionary from a perfect foresight consumer, we will change these parameters below
    TBS_dictionary = {
        'CRRA': 2.5,          # The dictionary includes our original parameters...
        'Rfree': 1.,
        'DiscFac': 1.,
        'LivPrb': [1.0],
        'PermGroFac': [1.0],
        'PermShkStd': [0.],   # no permanent shocks for our agent...    
        'PermShkCount': 1.,
        'TranShkStd': [0.],
        'TranShkCount': 0,
        'UnempPrb': 0.0,
        'IncUnemp': 0.,
        'BoroCnstArt': 0.0,
        'aXtraMin': 0.001,    # aXtra parameters specify how to construct the grid of assets.
        'aXtraMax': 0.5,      # Don't worry about these for now
        'aXtraNestFac': 3,
        'aXtraCount': 0,
        'aXtraExtra': [None],
        'vFuncBool': False,   # These booleans indicate whether the value function should be calculated
        'CubicBool': False,   # and whether to use cubic spline interpolation. You can ignore them.
        'aNrmInitMean' : 0.,
        'aNrmInitStd' : 0.0,  # These parameters specify the (log) distribution of normalized assets
        'pLvlInitMean' : 1.5, # and permanent income for agents at "birth". They are only relevant in
        'pLvlInitStd' : 0.,   # simulation and you don't need to worry about them.
        'PermGroFacAgg' : 0.,
        'T_retire': 0,        
        'UnempPrbRet': 0.0,   
        'IncUnempRet': 0.0, 
        'T_age' : None,
        'T_cycle' : 1,
        'cycles' : 0, #consumer is infinitely lived
        'AgentCount': 1,
        'tax_rate':0.0,
    }
    
    # Change the parameters we want
    MyTBStype = TractableConsumerType(**TBS_dictionary)
    MyTBStype.UnempPrb = Upsilon
    MyTBStype.DiscFac = Beta
    MyTBStype.Rfree = R
    MyTBStype.PermGroFac = G/(1-Upsilon)
    MyTBStype.CRRA = Rho

    try:
        MyTBStype.solve()
    except:
        print('Those parameter values violate a condition required for solution!')
    cE = MyTBStype.solution[0].cFunc(mE_range)*MyTBStype.PermGroFac#Cmp/(1-Upsilon)  # Employed consumption function
    cU = MyTBStype.solution[0].cFunc_U(mE_range)*MyTBStype.PermGroFac#Cmp/(1-Upsilon) # Unemployed consumption function  
    
    # Find approx growth rate of consumption
    Gamma = G/(1-Upsilon)
    cEDelLog = np.log(((R*Beta)**(1/Rho))*(1+Upsilon*((cE/cU)**Rho - 1))**(1/Rho))
    
    # Note: since we are using approximations for the growth rate, HARK mTarg is slightly different
    # from what we expect graphically. So for ease we note that mTarg occurs when the growth rate hits the gamma line
    # Find mTarg
    idx = np.nanargmin(np.abs(cEDelLog-np.log(Gamma)))
    mTarg = mE_range[idx]

    plt.figure(figsize=(9,6))
    plt.plot(mE_range, cEDelLog, 'k-', label='$\Delta log c^e_{t+1}$') # Plot c growth
    plt.hlines(np.log(G/(1-Upsilon)), mEMin, mEMax, colors='b', linestyles='solid', label='$\gamma$')
    plt.hlines(np.log(PatienceFactor(R, Beta, Rho, G, Upsilon)), mEMin, mEMax, colors='g', linestyles='dashed', label='$\\thorn$')
    plt.vlines(mTarg, -0.01*mEMax, 0.01*mEMax, colors='r', linestyles='dashed', label='$\\breve{m}^e$')
    
    plt.xlim(mEMin, mEMax)
    plt.ylim(-0.01*mEMax, 0.01*mEMax)
    plt.xlabel('$m^e_{t}$')
    plt.ylabel('$Growth$')
    plt.legend()
    plt.show()

    return None

# Define some functions that will serve as widgets in our interactive plot
R_widget3 = widgets.FloatSlider(
    min=1.,
    max=1.03,
    step=0.001,
    value=1.01,
    continuous_update=False,
    readout_format='.4f',
    description='$R$')

Beta_widget3 = widgets.FloatSlider(
    min=0.8,
    max=0.99,
    step=0.001,
    value=0.96,
    continuous_update=False,
    readout_format='.3f',
    description='$\\beta$')

Rho_widget3 = widgets.FloatSlider(
    min=0.1,
    max=3,
    step=0.01,
    value=1.1,
    continuous_update=False,
    readout_format='.3f',
    description='$\\rho$')

G_widget3 = widgets.FloatSlider(
    min=1.,
    max=1.02,
    step=0.001,
    value=1.0025,
    continuous_update=False,
    readout_format='.3f',
    description='$G$')

Upsilon_widget3 = widgets.FloatSlider(
    min=0.0001,
    max=0.01,
    step=0.0001,
    value=0.005,
    continuous_update=False,
    readout_format='.3f',
    description='$\mho$')

mEMax_widget3 = widgets.FloatText(
    value=10,
    step=0.1,
    description='$m^e max$',
    disabled=False)

mEMin_widget3 = widgets.FloatText(
    value=0.,
    step=0.1,
    description='$m^e min$',
    disabled=False)

# Make the widget
interact(Plot3,
         R = R_widget3,
         Beta = Beta_widget3,
         Rho = Rho_widget3,
         G = G_widget3,
         Upsilon = Upsilon_widget3,
         mEMax = mEMax_widget3,
         mEMin = mEMin_widget3
        );

#### The simulated effect of a rise in Beta

We can see by adjusting the above three graphs that a rise in Beta leads to a higher target level of market resources, $\breve{m}^e$, and a higher steady state consumption level.

However, the above charts don't show us the transition path.

In the cells below we follow a simple procedure:
1. Simulate the model to find at what point in the first "seed" of shocks, our agent transitions into the absorbing unemployed state
2. Take a period before that shock as the steady state for the unemployed consumer
3. Change the discount factor and re-solve the model
4. Simulate the model again, and plot the movement to the new steady state

In [ ]:
# First we need to check at what point a shock comes along that takes our agent into the absorbing (unemployed) state

# Copy a dictionary from a perfect foresight consumer, we will change these parameters below
TBS_dictionary = {
    'CRRA': 2.5,          # The dictionary includes our original parameters...
    'Rfree': 1.,
    'DiscFac': 1.,
    'LivPrb': [1.0],
    'PermGroFac': [1.0],
    'PermShkStd': [0.],   # no permanent shocks for our agent...    
    'PermShkCount': 1.,
    'TranShkStd': [0.],
    'TranShkCount': 0,
    'UnempPrb': 0.0,
    'IncUnemp': 0.,
    'BoroCnstArt': 0.0,
    'aXtraMin': 0.001,    # aXtra parameters specify how to construct the grid of assets.
    'aXtraMax': 0.5,      # Don't worry about these for now
    'aXtraNestFac': 3,
    'aXtraCount': 0,
    'aXtraExtra': [None],
    'vFuncBool': False,   # These booleans indicate whether the value function should be calculated
    'CubicBool': False,   # and whether to use cubic spline interpolation. You can ignore them.
    'aNrmInitMean' : 0.,
    'aNrmInitStd' : 0.0,  # These parameters specify the (log) distribution of normalized assets
    'pLvlInitMean' : 1.5, # and permanent income for agents at "birth". They are only relevant in
    'pLvlInitStd' : 0.,   # simulation and you don't need to worry about them.
    'PermGroFacAgg' : 0.,
    'T_retire': 0,        
    'UnempPrbRet': 0.0,   
    'IncUnempRet': 0.0, 
    'T_age' : None,
    'T_cycle' : 1,
    'cycles' : 0, #consumer is infinitely lived
    'AgentCount': 1,
    'tax_rate':0.0,
}
    
# Change the parameters we want to some arbitrary parameters that we know at least yield a solution
MyTBStype = TractableConsumerType(**TBS_dictionary)
MyTBStype.UnempPrb = 0.005
MyTBStype.DiscFac = 0.96
MyTBStype.Rfree = 1.01
MyTBStype.PermGroFac = 1.002/(1-0.005)
MyTBStype.CRRA = 1.1

try:
    MyTBStype.solve()
except:
    print('Those parameter values violate a condition required for solution!')        

# Simulate the economy so that we get to steady state
MyTBStype.T_sim = 500
MyTBStype.aLvlInitMean = 1.
MyTBStype.aLvlInitStd = 0.
MyTBStype.track_vars = ['mLvlNow','cLvlNow']
MyTBStype.initializeSim()
MyTBStype.simulate()

plt.figure(figsize=(9,6))
plt.plot(np.mean(MyTBStype.mLvlNow_hist, axis=1))
plt.xlabel('Simulated time period')
plt.ylabel('Average market resources mLvl')
plt.show()

plt.figure(figsize=(9,6))
plt.plot(np.mean(MyTBStype.cLvlNow_hist, axis=1))
plt.xlabel('Simulated time period')
plt.ylabel('Consumption level')
plt.show()

We can see that the first seed of shocks means that our agent becomes unemployed just after period 100, and they have reached a steady state by this point. So lets take period 100 as the steady state for the employed consumer. Now we follow our plan above to plot the impulse reponses.

In [ ]:
# Find and plot impulse responses

MyTBStype.T_sim = 100
MyTBStype.aLvlInitMean = 1.
MyTBStype.aLvlInitStd = 0.
MyTBStype.track_vars = ['mLvlNow','cLvlNow']
MyTBStype.initializeSim()
MyTBStype.simulate()

# Capture steady states
sscE = MyTBStype.cLvlNow
ssmE = MyTBStype.mLvlNow

# Now lets change the discount factor from 0.96 to 0.97
MyTBStype.DiscFac = 0.97
MyTBStype.solve() #remember to resolve the consumer's problem
MyTBStype.initializeSim()
MyTBStype.simulate()

# Organise the data into a impulse response plot
TimePeriod = np.arange(-20, 100, 1)

cE = np.zeros((120,1))
cE[:20,:] = sscE
cE[20:,:] = MyTBStype.cLvlNow_hist

mE = np.zeros((120,1))
mE[:20,:] = ssmE
mE[20:,:] = MyTBStype.mLvlNow_hist

plt.figure(figsize=(9,6))
plt.plot(TimePeriod, cE)
plt.xlabel('Time since change to Beta')
plt.ylabel('Consumption level for employed consumer')
plt.show()

plt.figure(figsize=(9,6))
plt.plot(TimePeriod, mE)
plt.xlabel('Time since change to Beta')
plt.ylabel('Market resources for employed consumer')
plt.show()